In [1]:
%load_ext autoreload
%autoreload 2


import tensorflow as tf
from tensorflow  import keras
import tensorflow_hub as hub
import numpy as np

import librosa 
from util import WavDataset
import matplotlib.pyplot as plt

In [33]:
import h5py
from config import *
from pathlib import Path

np.random.seed(0)

def chunk_generator():
    chunk_len = 5
    overlap = 1
    sr = 16_000
    hdf5_file = INTERMEDIATE / 'train.hdf5'

    chunk_len *= sr
    overlap *= sr
    with h5py.File(hdf5_file, 'r') as hdf5_dataset:
        
        # get chunk info and shuffle
        chunks = []
        for rec in list(hdf5_dataset):
            Y = hdf5_dataset[rec]['Y']     
            n_samples = Y.shape[1]
            chunks += [
                (rec, start, start + chunk_len) 
                for start in range(0, n_samples, chunk_len - overlap)
            ]
            
        np.random.shuffle(chunks)
    
        # generate chunks
        for random_chunk in chunks:
            rec, s, e = random_chunk
            Y = hdf5_dataset[rec]['Y']  
            samples = hdf5_dataset[rec]['X']
            s_slice = np.array(samples[s:e])
            y_slice = np.array(Y[:, s:e])
            
            def contract(segment):
                return np.array([
                    ss.mean() > 0.4 for ss in segment
                ])

            # https://www.kaggle.com/models/google/yamnet
            frame = int(1 * sr)
            hop = int(0.5 * sr)
            
            y_frames = np.array([
                contract(y_slice[:, i:i+frame]) for i in range(0, y_slice.shape[1] - (frame - hop) , hop)
            ]) # 9 frames 
            
            yield s_slice.T, y_frames # !! TODO veryfy that y is correct
            

raw_dataset = tf.data.Dataset.from_generator(
    chunk_generator,
    output_signature=(
        tf.TensorSpec(shape=(None), dtype=tf.float32),
        tf.TensorSpec(shape=(9, 4), dtype=tf.bool)))

for s in raw_dataset.take(1):
    print(s)

(<tf.Tensor: shape=(80000,), dtype=float32, numpy=
array([-0.00452962, -0.0067026 , -0.00624345, ..., -0.00081211,
       -0.00229448, -0.00023884], dtype=float32)>, <tf.Tensor: shape=(9, 4), dtype=bool, numpy=
array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])>)


In [37]:
_, emb, _ = YAMnet(np.zeros(16_000 * 3))
emb.shape

TensorShape([6, 1024])

In [28]:
yamnet_url = 'https://tfhub.dev/google/yamnet/1'
yamnet_layer = hub.KerasLayer(yamnet_url, input_shape=(None,), dtype=tf.float32, trainable=False)

22:51:24 INFO Using /tmp/tfhub_modules to cache modules.


In [36]:
def get_embeddings(sample_chunk, Y_chunk):
    _, emb, _ = yamnet_layer(sample_chunk)
    return emb, Y_chunk 

full_dataset = raw_dataset.map(lambda x, y: get_embeddings(x, y))

In [34]:
# inp = keras.Input(shape=(16_000 * 5))
# _, embeddings, _ = YAMnet(inp)
# output = keras.layers.Dense(4, activation='sigmoid')(embeddings)
# model = keras.Sequential(inputs=inp, outputs=output)

# model = tf.keras.Sequential([
#     yamnet_base,
#     # tf.keras.layers.GlobalAveragePooling1D(),  # Reduce the dimensionality
#     tf.keras.layers.Dense(4, activation='softmax')  # Output layer for classification
# ])

from keras import layers

inputs = layers.Input(shape=(9, 1024), dtype=tf.float32)
x = layers.Dense(256, activation='relu')(inputs)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(4, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  
    metrics=['accuracy']
)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9, 1024)]         0         
                                                                 
 dense_4 (Dense)             (None, 9, 256)            262400    
                                                                 
 dense_5 (Dense)             (None, 9, 128)            32896     
                                                                 
 dense_6 (Dense)             (None, 9, 64)             8256      
                                                                 
 dense_7 (Dense)             (None, 9, 4)              260       
                                                                 
Total params: 303,812
Trainable params: 303,812
Non-trainable params: 0
_________________________________________________________________


In [37]:
history = model.fit(full_dataset.batch(32), epochs=1)

InvalidArgumentError:  Incompatible shapes: [32,9] vs. [32,10]
	 [[node Equal
 (defined at /usr/local/lib/python3.8/dist-packages/keras/metrics.py:3566)
]] [Op:__inference_train_function_17788]

Errors may have originated from an input operation.
Input Source operations connected to node Equal:
In[0] ArgMax (defined at /usr/local/lib/python3.8/dist-packages/keras/metrics.py:3567)	
In[1] ArgMax_1:

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/home/ethan/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/home/ethan/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
>>>     app.start()
>>> 
>>>   File "/home/ethan/.local